In [1]:
import simpy
import numpy as np

# -----------------------------
# KONFIGURATION
# -----------------------------
WEEKS_PER_YEAR = 52
SIMULATION_DURATION = int(WEEKS_PER_YEAR * 3) # 3 Jahre
SEED = 42

# 1. NACHFRAGE (INPUT)
TOTAL_PATIENTS_YEAR = 95201
YES_PATIENTS_YEAR = 84557
P_START_THERAPY = YES_PATIENTS_YEAR / TOTAL_PATIENTS_YEAR 

# 2. ZEIT-PARAMETER (DAUER)
# (Muss VOR der Kapazitätsberechnung stehen, da wir die Dauer dafür brauchen)
SESSIONS_REQUIRED = 24  
WORKING_WEEKS = 42  # Abzug von Urlaub/Krankheit
TIME_STRETCH = 52 / WORKING_WEEKS 
REAL_DURATION_WEEKS = SESSIONS_REQUIRED * TIME_STRETCH # ca. 30 Wochen

# 3. KAPAZITÄT (AUTOMATISCH BERECHNET)
SLOTS_PART_TIME = 5490 * 9
SLOTS_FULL_TIME = 3510 * 19
THEORETICAL_CAPACITY = SLOTS_PART_TIME + SLOTS_FULL_TIME # ~116.100

# A) Wie viele Plätze bräuchten wir theoretisch für 100% Deckung?
# Formel: (Patienten pro Jahr * Dauer in Wochen) / 52 Wochen
REQUIRED_SLOTS_SATURATION = (YES_PATIENTS_YEAR * REAL_DURATION_WEEKS) / WEEKS_PER_YEAR

# B) Berechnung des "Break-Even" Faktors
# Bei diesem Faktor wäre Angebot == Nachfrage (genau 100%)
BREAK_EVEN_FACTOR = REQUIRED_SLOTS_SATURATION / THEORETICAL_CAPACITY

# C) Szenario-Einstellung (Der "Härtegrad")
# 1.00 = Exakt passend (Wartezeit entsteht nur durch Zufall)
# 0.95 = 5% zu wenig Plätze (Stau baut sich garantiert auf)
# 0.90 = 10% zu wenig Plätze (Stau explodiert)
SCENARIO_ADJUSTMENT = 0.95 

# D) Das Ergebnis für die Simulation
REAL_EFFICIENCY_FACTOR = BREAK_EVEN_FACTOR * SCENARIO_ADJUSTMENT
REAL_CAPACITY = int(THEORETICAL_CAPACITY * REAL_EFFICIENCY_FACTOR)


# 4. WACHSTUM
# Startrate basierend auf dem Gesamt-Traffic
INITIAL_WEEKLY_RATE = (TOTAL_PATIENTS_YEAR / WEEKS_PER_YEAR) * 1.0
GROWTH_PER_WEEK = 0.005 

stats_wait_times = []

# -----------------------------
# PROZESSLOGIK
# -----------------------------
def patient_process(env, name, therapy_slots, dropped_counter):
    t_ankunft = env.now
    # Filter: Will der Patient überhaupt?
    if np.random.random() > P_START_THERAPY:
        dropped_counter['count'] += 1
        return 

    with therapy_slots.request() as req:
        yield req 
        # Platz bekommen -> Wartezeit speichern
        stats_wait_times.append(env.now - t_ankunft)
        # Platz belegen für die reale Dauer (ca. 30 Wochen)
        yield env.timeout(REAL_DURATION_WEEKS)

def pre_fill_slot(env, therapy_slots, rng):
    with therapy_slots.request() as req:
        yield req
        # Zufällige Restdauer für den Start-Patienten
        yield env.timeout(rng.uniform(0, REAL_DURATION_WEEKS))

def setup(env, capacity, rng):
    therapy_slots = simpy.Resource(env, capacity=capacity)
    
    # Warm-up: Klinik komplett füllen
    for _ in range(capacity):
        env.process(pre_fill_slot(env, therapy_slots, rng))
        
    dropped_counter = {'count': 0}
    i = 0
    current_rate = INITIAL_WEEKLY_RATE

    while env.now < SIMULATION_DURATION:
        scale = 1.0 / current_rate
        interarrival_time = rng.exponential(scale=scale)
        yield env.timeout(interarrival_time)
        
        env.process(patient_process(env, f"Patient_{i}", therapy_slots, dropped_counter))
        i += 1
        # Exponentielles Wachstum der Nachfrage
        current_rate = current_rate * (1 + (GROWTH_PER_WEEK * interarrival_time))

# -----------------------------
# MAIN
# -----------------------------
if __name__ == "__main__":
    print(f"--- AUTOMATISCHE KAPAZITÄTS-BERECHNUNG ---")
    print(f"Nachfrage (Yes-Patients): {YES_PATIENTS_YEAR}")
    print(f"Reale Therapiedauer:      {REAL_DURATION_WEEKS:.2f} Wochen")
    print(f"Benötigte Plätze (100%):  {int(REQUIRED_SLOTS_SATURATION)}")
    print(f"Theoretische Plätze:      {THEORETICAL_CAPACITY}")
    print(f"-"*40)
    print(f"Break-Even Faktor:        {BREAK_EVEN_FACTOR:.4f} (Das wären 100%)")
    print(f"Szenario-Einstellung:     {SCENARIO_ADJUSTMENT*100:.1f}% des Bedarfs (Unterversorgung)")
    print(f"-> Genutzter Faktor:      {REAL_EFFICIENCY_FACTOR:.4f}")
    print(f"-> Reale Kapazität:       {REAL_CAPACITY} Slots")
    print(f"="*60)
    
    rng = np.random.default_rng(SEED)
    env = simpy.Environment()
    env.process(setup(env, capacity=REAL_CAPACITY, rng=rng))
    env.run(until=SIMULATION_DURATION)

    waits = np.array(stats_wait_times)
    n_started = len(waits)

    print(f"\n--- ERGEBNISSE ---")
    if n_started > 0:
        mean_weeks = np.mean(waits)
        median_weeks = np.median(waits)
        p90_weeks = np.quantile(waits, 0.90)

        print(f"Anzahl therapierte Patienten: {n_started}")
        print(f"Durchschnitt Wartezeit:       {mean_weeks*7:.1f} Tage")
        print(f"Median Wartezeit:             {median_weeks*7:.1f} Tage")
        print(f"P90 (Risiko):                 {p90_weeks*7:.1f} Tage")
        print(f"Max Wartezeit:                {np.max(waits)*7:.1f} Tage")
    else:
        print("Keine Daten.")

--- AUTOMATISCHE KAPAZITÄTS-BERECHNUNG ---
Nachfrage (Yes-Patients): 84557
Reale Therapiedauer:      29.71 Wochen
Benötigte Plätze (100%):  48318
Theoretische Plätze:      116100
----------------------------------------
Break-Even Faktor:        0.4162 (Das wären 100%)
Szenario-Einstellung:     95.0% des Bedarfs (Unterversorgung)
-> Genutzter Faktor:      0.3954
-> Reale Kapazität:       45902 Slots

--- ERGEBNISSE ---
Anzahl therapierte Patienten: 241014
Durchschnitt Wartezeit:       122.8 Tage
Median Wartezeit:             105.7 Tage
P90 (Risiko):                 269.0 Tage
Max Wartezeit:                317.6 Tage
